### Setup

In [11]:
import requests
import csv


ssl_cert_file = "../ssl/cert.pem"
url = "https://192.168.1.100:8000/"

username = "test"
file_username = "user4"

In [12]:
response = requests.get(url, verify=ssl_cert_file)
if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'Hello, World!'}


### Training
Remember to move file_username file from training directory (so its not included as negative example)

In [13]:
tsv_file_path = f"../datasets/{file_username}.tsv"
num_stages = 5

with open(tsv_file_path, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")
    headers = next(reader)
    rows = list(reader)


rows_per_stage = len(rows) // num_stages
for i in range(num_stages - 1):
    start_index = rows_per_stage * i
    end_index = start_index + rows_per_stage
    
    tsv_data = "\t".join(headers) + "\n"
    tsv_data += "\n".join(["\t".join(row) for row in rows[start_index:end_index]])
    
    response = requests.post(
        url + 'upload_tsv',
        params={"username": username},
        data=tsv_data,
        verify=ssl_cert_file,
    )

    if response.status_code == 200:
        print("Response from server:", response.json())
    else:
        print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}


In [14]:
tsv_data = "\t".join(headers) + "\n"
tsv_data += "\n".join(["\t".join(row) for row in rows[rows_per_stage * (num_stages - 1):]])

response = requests.post(
    url + 'train',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully. Training succeeded.'}


### Inference

In [15]:
tsv_file_path = f"../datasets/inference/{file_username}.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()


response = requests.post(
    url + 'inference',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:\n", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server:
 {'message': 'TSV data received successfully. Inference succeeded', 'score': 1.0, 'prediction': 1}


In [16]:
tsv_file_path = f"../datasets/inference/user1.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()


response = requests.post(
    url + 'inference',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:\n", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server:
 {'message': 'TSV data received successfully. Inference succeeded', 'score': 0.7887324094772339, 'prediction': 1}
